# Porządkowanie konfiguracji eksperymentów z biblioteką Hydra

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-f", "--fff", help="a dummy arg", default="1")

_StoreAction(option_strings=['-f', '--fff'], dest='fff', nargs=None, const=None, default='1', type=None, choices=None, required=False, help='a dummy arg', metavar=None)

## Instalacja bibliotek

In [ ]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 13.0 MB/s eta 0:00:00


In [ ]:
!pip install hydra-core --upgrade

## Wprowadzenie
Hydra to framework, który upraszcza proces zarządzania plikami konfiguracyjnymi. Pliki konfiguracyjne umożliwiają zapisywanie hiperparametrów modelu, globalnych stałych, czy określenia podziału zbiorów danych. Pozwalają wprosty sposób zarządzać eksperymentami bez ingerencji w kod.

W odróżnieniu od poprzednich zajęć, dzisiaj będziemy pracować ze zwykłymi plikami .py.

1.1 Utwórz katalog 'config' i umieść w nim pusty plik 'basic_config.yaml'. Plik config.yaml uzupełnij poniższym tekstem:

```
animal: dog
name: Max
```

1.2. Utwórz plik basic.py i umieść w nim poniższy kod:
```
import os
import hydra
# omegaconf instaluje się razem z hydrą
# używamy go tylko do type annotation argumentu cfg
from omegaconf import DictConfig #
import argparse
from hydra import initialize, compose
# dekorator podpinający config do funkcji
@hydra.main(config_path="config", config_name="config", version_base='1.1')
def animal_info(cfg: DictConfig):
    working_dir = os.getcwd()
    print(f"The current working directory is {working_dir}")
    # Wyciągamy elementy configu
    print(f"This is a {cfg.animal}")
    print(f"It's name is {cfg.name}.")
if __name__=='__main__':
  animal_info()
```

1.3. Uruchom plik poniższą komendą:

In [ ]:
!python basic.py

/usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
The current working directory is /content/outputs/2023-12-03/20-15-53
This is a dog
It's name is Max.


Możemy zauważyć, że funkcja wyciągnęła poprawne informacje z configu. W ścieżce folderów utworzył się nowy folder 'outputs', który zawiera plik konfiguracyjny z danego eksperymentu oraz logi.

1.4 Do pliku 'outputs' możemy zapisywać również wyniki. Dodaj poniższy kod do funkcji animal info, aby wyświetlić zapisać o zwierzęciu.

```
orig_cwd = hydra.utils.get_original_cwd()
# Zapisz informacje do pliku
path = f"output.txt"
with open(path, "w") as f:
    f.write(f"This is a {cfg.animal}")
    f.write(f"It's name is {cfg.name}.")
```

## Hydra w uczeniu maszynowym

2.1 W pliku config utwórz nowy plik o nazwie 'ml_config.yaml'.

2.2 W pliku config umieść trzy parametry o wybranych wartościach: batch_size, learning_rate, hidden_dim

2.3 Utwórz funkcję do treningu modelu korzystając z ml_config oraz poniższego kodu

In [ ]:
import os
import torch
import yaml
import hydra
from omegaconf import DictConfig
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

from torchvision.datasets.mnist import MNIST
from torchvision import transforms

from pytorch_lightning.loggers import CSVLogger

In [ ]:
class LitClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters(config) # <- zapisujemy hiperparametry (wszystkie zmienne z funkcji init)

        self.l1 = torch.nn.Linear(28 * 28, self.hparams.hidden_dim) # <- możemy potem skorzystać ze zmiennej  self.hparams
        self.l2 = torch.nn.Linear(self.hparams.hidden_dim, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('valid_loss', loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)

        with open('results.txt', 'a') as f:
            f.write(f'Batch: {batch_idx}, Loss: {loss.item()}\n')

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

In [ ]:
with open('/content/config/ml_config.yaml', 'r') as f:
        config = yaml.safe_load(f)

def train_model(config):
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    hidden_dim = config['hidden_dim']

    dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
    mnist_test = MNIST('', train=False, download=True, transform=transforms.ToTensor())
    mnist_train, mnist_val = random_split(dataset, [55000, 5000])
    train_loader = DataLoader(mnist_train, batch_size=batch_size, num_workers=os.cpu_count())
    val_loader = DataLoader(mnist_val, batch_size=batch_size, num_workers=os.cpu_count())
    test_loader = DataLoader(mnist_test, batch_size=batch_size, num_workers=os.cpu_count())

    model = LitClassifier(config)

    logger = pl.loggers.CSVLogger('logs', name='LitClassifier')

    trainer = pl.Trainer(max_epochs=5, logger=logger)
    trainer.fit(model, train_loader, val_loader)
    trainer.test(dataloaders=test_loader)

In [ ]:
if __name__ == "__main__":
    train_model(config)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 100 K 
1 | l2   | Linear | 1.3 K 
--------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at logs/LitClassifier/version_0/checkpoints/epoch=4-step=8595.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at logs/LitClassifier/version_0/checkpoints/epoch=4-step=8595.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.8512787818908691     │
└───────────────────────────┴───────────────────────────┘

2.4 Zapisz wynik ze zbioru testowego do pliku 'results.txt'.

2.5 Zapisz logowane lossy do pliku csv korzystając z odpowiedniego loggera z pytorch lightning. Wyniki powinny znaleźć się w pliku outputs.

## Multirun - sprawdzanie treningu przy różnych wartościach parametrów przy pomocy jednej komendy

Zapisz wyniki treningów przy 3 różnych kombinacjach hiperparametrów. Tak modyfikuj config, aby wystarczyło wywołać funkcję do treningu tylko raz.

Sprawdź dokumentację: https://hydra.cc/docs/tutorials/basic/running_your_app/multi-run

In [ ]:
!python multirun.py -m

[2023-12-03 22:23:26,312][HYDRA] Launching 1 jobs locally
[2023-12-03 22:23:26,312][HYDRA] 	#0 : 
/usr/local/lib/python3.10/dist-packages/hydra/_internal/core_plugins/basic_launcher.py:74: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
100% 9912422/9912422 [00:00<00:00, 106821591.71it/s]
Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw

100% 28881/28881 [00:00<00:00, 158554573.07it/s]
Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw

100% 1648877/1648877 [00:00<00:00, 35857787.09it/s]
Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw

100% 4542/4542 [00:00<00:00, 30677179.98it/s]
Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw

Error executing job with overrides: []
Traceback (most recent call last):
  File "/content/multirun.py", line 59, in train_model
    train_loader 